In [2]:
# importing the requests library
import requests

### Data Gathering

In [86]:
output_subquery = """
                    ?time{0} wikibase:apiOutput "revisions/rev[{0}]/@timestamp" . 
                    ?comment{0} wikibase:apiOutput "revisions/rev[{0}]/@comment" .
                    ?revision{0} wikibase:apiOutput "revisions/rev[{0}]/@revid".
                    """
output_subquery = "".join([output_subquery.format(i) for i in range(1,16)])

query = """
        SELECT * {{
         SERVICE wikibase:mwapi {{
          bd:serviceParam wikibase:endpoint "www.wikidata.org" .
          bd:serviceParam wikibase:api "Generator" .
          bd:serviceParam mwapi:generator "revisions" .
          bd:serviceParam mwapi:titles "Property:{0}" .
          bd:serviceParam mwapi:grvprop "timestamp|comment" .
          bd:serviceParam mwapi:grvlimit "15".
          bd:serviceParam mwapi:prop  "revisions". 
             {1}
          }}
        }}
        order by ?time     
        """.format('P856',output_subquery)

In [87]:
# api-endpoint
URL = "https://query.wikidata.org/sparql"
 
# default format
default_format = 'json'
 
# defining a params dict for the parameters to be sent to the API
PARAMS = {'query':query,
          'format':default_format,
          'output':default_format,
          'results':default_format}
 
# sending get request and saving the response as response object
r = requests.get(url = URL,params = PARAMS)
 
# extracting data in json format
data = r.json()

### Data parsing

In [106]:
# comment#, time#, revision#
data['results']['bindings'][0]['comment10']['value']

'/* wbsetdescription-add:1|ht */ URL sit ofisyèl bagay la'

Edit message for changes related to labels, descriptions and aliases of Wikidata
properties

|Attribute |Add| Update |Remove|
|---|---|---|---|
|label |wbsetlabel-add| wbsetlabel-set| wbsetlabel-remove|
|description |wbsetdescription-add| wbsetdescription-set| wbsetdescription-remove|
|alias |wbsetaliases-add |wbsetaliases-add-remove| wbsetaliases-remove|

In [82]:
#data
#
#
#
data

{'head': {'vars': ['time1',
   'comment1',
   'revision1',
   'time2',
   'comment2',
   'revision2',
   'time3',
   'comment3',
   'revision3',
   'time4',
   'comment4',
   'revision4',
   'time5',
   'comment5',
   'revision5',
   'time6',
   'comment6',
   'revision6',
   'time7',
   'comment7',
   'revision7',
   'time8',
   'comment8',
   'revision8',
   'time9',
   'comment9',
   'revision9',
   'time10',
   'comment10',
   'revision10',
   'time11',
   'comment11',
   'revision11',
   'time12',
   'comment12',
   'revision12',
   'time13',
   'comment13',
   'revision13',
   'time14',
   'comment14',
   'revision14',
   'time15',
   'comment15',
   'revision15']},
 'results': {'bindings': [{'comment14': {'type': 'literal',
     'value': '/* wbremoveclaims-remove:1| */ [[Property:P1629]]: [[Q22137024]]'},
    'comment13': {'type': 'literal',
     'value': '/* wbsetclaim-update:2||1|2 */ [[Property:P2302]]: [[Q19474404]]'},
    'time14': {'type': 'literal', 'value': '2018-08-21T1